In [4]:
import pandas as pd
import os
from tqdm import tqdm
import numpy as np
from sklearn.preprocessing import LabelEncoder

data_folder = '/home/thebatou/Documents/Uni_Potsdam/Competitive_Data_Analysis/data/'
print 'Loading data ...'
data = pd.read_csv(os.path.join(data_folder, 'data_num_bit.csv'), index_col=0)
print 'Done loading data! {} data were loaded.'.format(data.shape[0])

Loading data ...
Done loading data! 2177 data were loaded.


In [5]:
#data.ix[0]
data

,loan_identifier,nominal_interest_rate,term,purpose,project_description,borrower_identifier,borrower_rating,employment,region,facebook,...,fraudulent,address_lat,address_lng,net_income_cents,currency,rates_count,rates_paid,base_currency,location,prices
0,E4ESQ31PV,1115,term_6_weeks,investment,I need to make some investments but need to co...,18TFGM41GTH,B,self_employed,DE,none,...,False,52.540658,13.417918,320000,EUR,1,1,btc,Germany,2158.125468
1,E4EVG31PW,1115,term_6_weeks,renovation,This is my third reputation loan. Of course th...,18TFGM41GTH,B,self_employed,DE,none,...,False,52.540658,13.417918,320000,EUR,1,1,btc,Germany,2158.125468
2,E4KSS31PZ,770,term_6_months,education,I want to start using bitcoin. This bitcoin lo...,18TFGM41GTJ,A,self_employed,SK,connected,...,False,52.510964,13.304994,185000,EUR,6,6,btc,Slovakia,678.130952
3,E4MST31Q6,770,term_6_months,education,test loan,18TFGM41GTJ,A,self_employed,SK,connected,...,False,52.510964,13.304994,185000,EUR,6,6,btc,Slovakia,678.130952
4,E58B131QM,1115,term_6_weeks,other,reputation loan number 1,18TFGM41GTH,B,self_employed,DE,none,...,False,52.540658,13.417918,320000,EUR,1,1,btc,Germany,2158.125468
5,E58RN31QR,1029,term_6_months,renovation,reputation loan number 2,18TFGM41GTH,B,self_employed,DE,none,...,False,52.540658,13.417918,320000,EUR,6,6,btc,Germany,2158.125468
6,EBZYT31R7,766,term_6_weeks,investment,This is a project for you guys to get started.,18TFGM41GTH,A,self_employed,DE,none,...,False,52.540658,13.417918,320000,EUR,0,0,btc,Germany,2158.125468
7,EFZ3731SG,2457,term_36_months,investment,I work as a teacher and need a motorcycle to g...,EFYAG31SE,E,salaried,IT,none,...,False,45.458525,9.164262,134500,EUR,0,0,btc,Italy,1562.285808
8,EG08A31SK,2108,term_36_months,investment,I have a garage in Dijon. We are specialized o...,EFZQC31SJ,D,self_employed,FR,none,...,False,47.225268,5.137244,231400,EUR,0,0,btc,France,1937.240671
9,EH8MV31SR,766,term_6_weeks,consumption,"Test project, will be cancelled",18TFGM41GTJ,A,self_employed,SK,connected,...,False,52.510964,13.304994,185000,EUR,0,0,btc,Slovakia,678.130952


In [6]:
data.project_description.ix[13]

'Hey everyone, this is a reputation project. Give it a try to see how everything works!'

In [7]:
data.loc[835,'region'] = 'NA'
data.loc[1232,'region'] = 'NA'
data.loc[1839,'region'] = 'NA'
data.loc[1993,'region'] = 'NA'
#data['funded'] = data['funded_at'] != '-1'

In [8]:
for status in np.unique(data['status']):
    print '{} has {} representatives'.format(status, data[data['status']==status].shape[0])

canceled has 881 representatives
charged_off has 119 representatives
current has 59 representatives
defaulted has 49 representatives
expired has 535 representatives
fully_paid has 391 representatives
funded has 73 representatives
in_funding has 11 representatives
late_30 has 10 representatives
late_90 has 49 representatives


In [9]:
categorical_feature = ['loan_identifier', 'borrower_identifier', 'term', 'purpose', 'borrower_rating', 'employment', 'region', 'facebook', 'twitter',
                       'paypal', 'ebay', 'linkedin', 'currency', 'base_currency', 'location']#, 'funded', 'status']
transformed_data = pd.DataFrame()
encoders = []
for feature in categorical_feature:
    le = LabelEncoder()
    encoders.append(le)
    le.fit(np.unique(data[feature]).tolist())
#    print list(le.classes_)
    s = pd.Series(le.transform(data[feature].tolist()))
    s.name = feature
    transformed_data = pd.concat([transformed_data, s], axis=1)

/home/thebatou/anaconda/lib/python2.7/site-packages/numpy/lib/arraysetops.py:200: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  flag = np.concatenate(([True], aux[1:] != aux[:-1]))


In [10]:
from datetime import datetime
from pandas import to_datetime
from datetime import timedelta
# Extracting publication date
published_date = data['published_at']
published_date = published_date.str.rsplit(' ', expand=True, n=1)
published_date = published_date.drop(1, axis=1)
# Exctracting funding date
funded_date = data['funded_at']
funded_date = funded_date.str.rsplit(' ', expand=True, n=1)
funded_date = funded_date.drop(1, axis=1)
funded_date[funded_date[0]=='-1'] = pd.Timestamp.max.strftime('%Y-%m-%d %H:%M:%S')

published_date = to_datetime(published_date[0], format='%Y-%m-%d %H:%M:%S')
funded_date = to_datetime(funded_date[0], format='%Y-%m-%d %H:%M:%S')
diff_date_pub_fund = funded_date - published_date
#print diff_date_pub_fund[diff_date_pub_fund>timedelta(days=90000)].shape

diff_date_pub_fund = diff_date_pub_fund.dt.days

data['time_between_published_funded'] = diff_date_pub_fund
transformed_data['time_between_published_funded'] = diff_date_pub_fund

In [11]:
data['ratio_requested_funded'] = data['amount_funded']/data['amount_requested']
transformed_data['ratio_requested_funded'] = data['amount_funded']/data['amount_requested']
transformed_data['amount_funded'] = data['amount_requested']
transformed_data['amount_funded'] = data['amount_funded']
transformed_data['address_lat']= data['address_lat']
transformed_data['address_lng']= data['address_lng']
transformed_data['net_income_cents']= data['net_income_cents']
transformed_data['rates_paid']= data['rates_paid']
transformed_data['rates_count']= data['rates_count']
transformed_data['prices']= data['prices']
transformed_data['portion_average_salary'] = data['net_income_cents']/data['prices']
transformed_data['nominal_interest_rate']= data['nominal_interest_rate']
transformed_data['status']= data['status']
transformed_data.shape

(2177, 27)

In [12]:
print data.shape
print data.columns
print ''
print transformed_data.ix[10]

(2177, 32)
Index([u'loan_identifier', u'nominal_interest_rate', u'term', u'purpose',
       u'project_description', u'borrower_identifier', u'borrower_rating',
       u'employment', u'region', u'facebook', u'twitter', u'paypal', u'ebay',
       u'linkedin', u'published_at', u'funded_at', u'status',
       u'amount_requested', u'amount_funded', u'issuer_id', u'fraudulent',
       u'address_lat', u'address_lng', u'net_income_cents', u'currency',
       u'rates_count', u'rates_paid', u'base_currency', u'location', u'prices',
       u'time_between_published_funded', u'ratio_requested_funded'],
      dtype='object')

loan_identifier                      2088
borrower_identifier                   485
term                                    3
purpose                                 6
borrower_rating                         5
employment                              3
region                                 14
facebook                                1
twitter                                 1
pa

In [86]:
transformed_data.to_csv(path_or_buf='/home/thebatou/Documents/Uni_Potsdam/Competitive_Data_Analysis/transformed_data.csv', encoding='utf-8')